<a href="https://colab.research.google.com/github/MegatronRobot/FastAI_Practical_DL_for_coders_Coursefiles/blob/main/Getting_Started/is_it_bird_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>